In [4]:
import pandas as pd
from PIL import Image
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers
from tqdm import tqdm
from sklearn.model_selection import train_test_split

### Caricamento delle immagini in input in formato array numpy

In [5]:
# IMAGE_PATH_FOLDER = './Dati Sensori/Dataset Sensori Lavorato/1_M/1_M_1'
IMAGE_PATH_FOLDER = '../../Nuovi Dati/DatasetDistorti'

# Specificare solamente una tipologia di speckle
DISTORSIONI = ['DatasetPolvere-c']
GRADI_DISTORSIONE = 5

HEIGHT = 512
WIDTH = 512

TARGET_SIZE = (WIDTH, HEIGHT)
CHANNELS = 1 # Scala di grigi

In [6]:
def get_sotto_cartelle(path):
    return [x[1] for x in os.walk(path)][0]

def get_immagini_cartella(path_cartella):
    immagini_cartella = []
    cartelle_soggetti = get_sotto_cartelle(path_cartella)
    for soggetto in tqdm(cartelle_soggetti):
        path_soggetto = f'{path_cartella}/{soggetto}'
        cartelle_acquisizioni = get_sotto_cartelle(path_soggetto)
        for acquisizione in cartelle_acquisizioni:
            path_acquisizione = f'{path_soggetto}/{acquisizione}'
            for filename in os.listdir(path_acquisizione):
                if (filename.endswith(".jpg")):    
                    path_file = f'{path_acquisizione}/{filename}'
                    immagine = get_immagine_da_path(path_file)
                    immagini_cartella.append(immagine)
    return immagini_cartella
                    
def get_immagine_da_path(path_file):
    image = Image.open(path_file).convert('L')
    resized_image = image.resize(TARGET_SIZE)
    image_array = np.array(resized_image)
    normalized_image_array = image_array / 255.0
    return normalized_image_array


In [7]:
image_list = []
label_list = []
LISTA_DISTORSIONI = [0, 2, 3, 4]

for distorsione in DISTORSIONI:
    path_distorsione = f'{IMAGE_PATH_FOLDER}/{distorsione}'
    for grado_distorsione in LISTA_DISTORSIONI:
        label = 100 - float(25 * (grado_distorsione))
        path_cartella_grado = f'{path_distorsione}{grado_distorsione}'
        immagini_grado = get_immagini_cartella(path_cartella_grado)
        image_list.extend(immagini_grado)
        label_to_append = [label for x in range(len(immagini_grado))] 
        label_list.extend(label_to_append)
        

100%|██████████| 719/719 [00:03<00:00, 232.84it/s]


In [8]:
train_images = np.array(image_list)
train_weights = np.array(label_list)

In [9]:
X = train_images
y = train_weights

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [11]:
model = tf.keras.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(HEIGHT, WIDTH, CHANNELS)),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(1)  
])

Metal device set to: Apple M1 Pro


2023-07-25 16:05:10.032228: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-07-25 16:05:10.033522: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [12]:
model.compile(optimizer='adam', loss='mae', metrics = ['mae'])

In [13]:
model.fit(x=X_train, y=y_train, batch_size=1, epochs=10)

Epoch 1/10


2023-07-25 16:05:31.438213: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2023-07-25 16:05:31.841064: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


2300/2300 [==============================] - 249s 106ms/step - loss: 31.8341 - mae: 31.8341
Epoch 2/10
2300/2300 [==============================] - 244s 106ms/step - loss: 31.3079 - mae: 31.3079
Epoch 3/10
2300/2300 [==============================] - 245s 106ms/step - loss: 31.0458 - mae: 31.0458
Epoch 4/10
2300/2300 [==============================] - 244s 106ms/step - loss: 30.4249 - mae: 30.4249
Epoch 5/10
2300/2300 [==============================] - 244s 106ms/step - loss: 27.1379 - mae: 27.1379
Epoch 6/10
2300/2300 [==============================] - 244s 106ms/step - loss: 18.4047 - mae: 18.4047
Epoch 7/10
2300/2300 [==============================] - 244s 106ms/step - loss: 15.5586 - mae: 15.5586
Epoch 8/10
2300/2300 [==============================] - 244s 106ms/step - loss: 14.8790 - mae: 14.8790
Epoch 9/10
 727/2300 [========>.....................] - ETA: 2:52 - loss: 14.4163 - mae: 14.4163


KeyboardInterrupt



In [ ]:
predictions = model.predict(X_test)

In [ ]:
predictions_list = [i[0] for i in predictions] 

In [ ]:
clamp_pred = [min(100,i) for i in predictions_list]
clamp_pred = [max(0,i) for i in clamp_pred]

In [ ]:
max(abs(clamp_pred  - y_test))

In [ ]:
def avg(lst):
    return sum(lst) / len(lst)

avg(abs(clamp_pred - y_test))

In [26]:
a = (clamp_pred - y_test)

In [43]:
clamp_pred[310:320]

[75.10758,
 69.49355,
 100,
 19.621279,
 26.603579,
 84.98515,
 78.70568,
 44.255188,
 88.8831,
 1.7798048]

In [42]:
y_test[310:320]

array([ 75.,  50., 100.,  25.,  25.,  75.,  75.,  50., 100.,   0.])

In [44]:
# for i, el in enumerate(list(a)):
#     print(f'{i}: {el}')

In [ ]:
model.save('../../Modelli-v2/modello_pesi_inizializzati_polvere')